In [1]:
!nvidia-smi

Wed Jul  7 07:33:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#config consts
DATASET =     {'train':'train.tar.gz',
               'test':'test.tar.gz',
               'train_mask':'train_mask.tar.gz'}

METADATA=     {'image_level':'train_image_level.csv',
               'study_level':'train_study_level.csv',
               'df_meta':'df_meta.csv',
               'df_fold_rand830':'df_fold_rand830.csv',
               'train_dup':'duplicate.txt',
               'sample_sub':'sample_submission.csv'}

CFGMODEL_DIR_DICT = {'B3_512':'effb3-full-512-mask-v8/',
                     'B3_512_PNG': 'effb3-512-png-mask/',
                     'B4_512':'effb4-full-512-mask/',
                     'B5_600':'effb5-600-mask/',
                     'B5_640':'effb5-640-mask/',
                     'B5_640_M2': 'effb5-640-mask/',
                     'D121_640':'d121-640-mask/',
                     'D201_640':'d201-640-mask'}

INPUT_DIR ='/content/drive/My\ Drive/kaggle/covid19-det/input/'

OUTPUT_DIR = {'BSL':'/content/drive/My\ Drive/kaggle/covid19-det/output/',
              'NORM':'/content/drive/My Drive/kaggle/covid19-det/output/'}

IMPORT_DIR = '/content/drive/My Drive/kaggle/covid19-det/nbs/py/'

HENGCK_IM_DIR=IMPORT_DIR+'hengck_code/dummy_01q/'

WORK_DIR='/content/'

DATASET_DIR_DICT = {'256': INPUT_DIR+'256_jpg/',
                    '512': INPUT_DIR+'512_jpg/',
                    '512_PNG': INPUT_DIR+'512_png/',
                    '600': INPUT_DIR+'600_jpg/',
                    '640': INPUT_DIR+'640_jpg/',
                    '640_M2':INPUT_DIR+'640_jpg_m2/'}

EXPERIMENT='SZ640_D201_MASK_0FOLD_WL4'
EXPERIMENT_DIR = OUTPUT_DIR['BSL'] + EXPERIMENT+'/'
CFGMODEL_DIR = CFGMODEL_DIR_DICT['D201_640']
MASK_SIZE=(20,20)
DATASET_DIR = DATASET_DIR_DICT['640']
METADATA_DIR = INPUT_DIR+'metadata/'
FOLDS_SET=[0]
INITIAL_CHECKPOINTS=[None for i in range(5)]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pydicom
!pip install madgrad
!pip install timm

import sys
sys.path.append(HENGCK_IM_DIR)
sys.path.append(HENGCK_IM_DIR+CFGMODEL_DIR)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from common import *

from siim import *

# from lib.net.lookahead import *
# from lib.net.radam import *
from madgrad import MADGRAD

from model import *
from dataset import *

     |████████████████████████████████| 1.9MB 5.2MB/s 
     |████████████████████████████████| 378kB 4.1MB/s 


In [5]:
def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print('mkdir',work_dir+record)
    !mkdir {work_dir+record}
    print ('tar -xzf',work_dir+ds_dict[record],'-C',work_dir+record)
    !tar -xzf  {work_dir+ds_dict[record]} -C {work_dir+record}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}
copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
!ls /content/

copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train.tar.gz  to /content/
mkdir /content/train
tar -xzf /content/train.tar.gz -C /content/train
rm  /content/train.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/test.tar.gz  to /content/
mkdir /content/test
tar -xzf /content/test.tar.gz -C /content/test
rm  /content/test.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train_mask.tar.gz  to /content/
mkdir /content/train_mask
tar -xzf /content/train_mask.tar.gz -C /content/train_mask
rm  /content/train_mask.tar.gz
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_image_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_study_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_meta.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_fold_rand830.csv  to  /content/
copy  /content/drive/My\ D

In [6]:
!mkdir {EXPERIMENT_DIR}

mkdir: cannot create directory ‘/content/drive/My Drive/kaggle/covid19-det/output/SZ640_D201_MASK_0FOLD_WL4/’: File exists


In [7]:
#----------------
import torch.cuda.amp as amp

class AmpNet(Net):
    @torch.cuda.amp.autocast()
    def forward(self,*args):
        return super(AmpNet, self).forward(*args)

is_mixed_precision = True  #True #False
run_check_net()

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


image:  torch.Size([2, 3, 640, 640])
logit:  torch.Size([2, 4])
mask:  torch.Size([2, 1, 20, 20])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
#----------------
def train_augment(r):
    image = r['image']
    mask = r['mask']
    # if image[:2].shape != (image_size, image_size):
    #     image = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_AREA)

    if 1:
        for fn in np.random.choice([
            lambda image, mask : do_random_scale(image, mask, mag=0.20),
            lambda image, mask : do_random_stretch_y(image, mask, mag=0.20),
            lambda image, mask : do_random_stretch_x(image, mask, mag=0.20),
            lambda image, mask : do_random_shift(image, mask, mag=int(0.20*image_size)),
            lambda image, mask : (image, mask)
        ],1):
            image, mask = fn(image, mask)

        for fn in np.random.choice([
            lambda image, mask : do_random_rotate(image, mask, mag=15),
            lambda image, mask : do_random_hflip(image, mask),
            lambda image, mask : (image, mask)
        ],1):
            image, mask = fn(image, mask)

        # ------------------------
        for fn in np.random.choice([
            lambda image : do_random_intensity_shift_contast(image, mag=[0.5,0.5]),
            lambda image : do_random_noise(image, mag=0.05),
            lambda image : do_random_guassian_blur(image),
            lambda image : do_random_blurout(image, size=0.25, num_cut=2),
            #lambda image : do_random_clahe(image),
            #lambda image : do_histogram_norm(image),
            lambda image : image,
        ],1):
            image = fn(image)

    r['image'] = image
    r['mask'] = mask
    return r
    
def do_valid(net, valid_loader):

    valid_probability = []
    valid_truth = []
    valid_num = 0

    net.eval()
    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image = batch['image'].cuda()
        onehot = batch['onehot']
        label = onehot.argmax(-1)

        with torch.no_grad():
            #with amp.autocast():
                logit, mask = data_parallel(net,image)
                probability = F.softmax(logit,-1)

        valid_num += batch_size
        valid_probability.append(probability.data.cpu().numpy())
        valid_truth.append(label.data.cpu().numpy())
        print('\r %8d / %d  %s'%(valid_num, len(valid_loader.dataset),time_to_str(timer() - start_timer,'sec')),end='',flush=True)

    assert(valid_num == len(valid_loader.dataset))
    #print('')
    #----------------------
    truth = np.concatenate(valid_truth)
    probability = np.concatenate(valid_probability)
    predict = probability.argsort(-1)[:,::-1]

    loss = np_loss_cross_entropy(probability,truth)
    topk = (predict==truth.reshape(-1,1))
    acc  = topk[:, 0]
    topk = topk.mean(0).cumsum()
    acc = [acc[truth==i].mean() for i in range(num_study_label)]

    #---
    map  = np_metric_map_curve_by_class(probability, truth)*(4/6)

    return [loss, map.mean(), topk[0], topk[1]]

In [12]:
INITIAL_CHECKPOINTS=['best_model.pth' for i in range(5)]

In [13]:
INITIAL_CHECKPOINTS

['best_model.pth',
 'best_model.pth',
 'best_model.pth',
 'best_model.pth',
 'best_model.pth']

In [14]:
# start here ! ###################################################################################
def run_train():
    for fold in FOLDS_SET:
        out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d-fine'%fold
        
        initial_checkpoint=None
        if INITIAL_CHECKPOINTS[fold] is not None: 
          initial_checkpoint = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d/checkpoint/'%fold+INITIAL_CHECKPOINTS[fold]

        best_map = 0.3
        start_lr   = 0.0001#1
        batch_size = 8 #14 #22

        num_iteration = 8000
        iter_log    = 200
        iter_valid  = 200
        iter_save   = list(range(0, num_iteration+1, 200))
        a_iter_save = []

        ## setup  ----------------------------------------
        for f in ['checkpoint', 'train', 'valid', 'backup']: os.makedirs(out_dir + '/' + f, exist_ok=True)
        # backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.train.%s.zip'%IDENTIFIER)

        log = Logger()
        log.open(out_dir + '/log.train.txt', mode='a')
        log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
        log.write('\t%s\n' % COMMON_STRING)
        log.write('\texpirement = %s\n' % EXPERIMENT)
        log.write('\tout_dir  = %s\n' % out_dir)
        log.write('\n')

        ## dataset ------------------------------------
        df_train, df_valid = make_fold('train-%d'%fold)
        train_dataset = SiimDataset(df_train, train_augment)
        valid_dataset = SiimDataset(df_valid, )

        train_loader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size,
            drop_last   = True,
            num_workers = 2,
            pin_memory  = True,
            worker_init_fn=lambda id: np.random.seed(torch.initial_seed() // 2 ** 32 + id),
            collate_fn  = null_collate,
        )
        valid_loader  = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size  = 16,
            drop_last   = False,
            num_workers = 2,
            pin_memory  = True,
            collate_fn  = null_collate,
        )

        log.write('train_dataset : \n%s\n'%(train_dataset))
        log.write('valid_dataset : \n%s\n'%(valid_dataset))
        log.write('\n')


        ## net ----------------------------------------
        log.write('** net setting **\n')
        if is_mixed_precision:
            scaler = amp.GradScaler()
            net = AmpNet().cuda()
        else:
            net = Net().cuda()


        if initial_checkpoint is not None:
            f = torch.load(initial_checkpoint, map_location=lambda storage, loc: storage)
            start_iteration = f['iteration']
            num_iteration = start_iteration + 8000
            start_epoch = f['epoch']
            state_dict  = f['state_dict']
            net.load_state_dict(state_dict,strict=True)  #True
        else:
            start_iteration = 0
            start_epoch = 0


        log.write('net=%s\n'%(type(net)))
        log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
        log.write('\n')

        # -----------------------------------------------
        if 0: ##freeze
            for p in net.block0.backbone.parameters(): p.requires_grad = False


        #optimizer = Lookahead(RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr), alpha=0.5, k=5)
        #optimizer = RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr)
        optimizer = MADGRAD( filter(lambda p: p.requires_grad, net.parameters()), lr=start_lr, momentum= 0.9, weight_decay= 0, eps= 1e-06)


        # num_iteration = 8000
        # iter_log    = 100
        # iter_valid  = 100
        # iter_save   = list(range(0, num_iteration, 100))#1*1000

        log.write('optimizer\n  %s\n'%(optimizer))
        log.write('\n')


        ## start training here! ##############################################
        log.write('** start training here! **\n')
        log.write('   fold = %d\n'%(fold))
        log.write('   is_mixed_precision = %s \n'%str(is_mixed_precision))
        log.write('   batch_size = %d\n'%(batch_size))
        log.write('   experiment = %s\n' % str(EXPERIMENT.split('/')[-2:]))
        log.write('                      |-----------VALID-------------|-----TRAIN/BATCH -----|------TIME-----\n')
        log.write('rate     iter   epoch | loss    map   topk0  topk1  | loss0  loss1  loss2  |               \n')
        log.write('-------------------------------------------------------------------------------------------\n')
                  #0.00000  0.00*  0.00  | 0.000  0.000  0.000  0.000  | 0.000  0.000  0.000  |  0 hr 00 min

        def message(mode='print'):
            if mode==('print'):
                asterisk = ' '
                loss = batch_loss
            if mode==('log'):
                asterisk = '*' if iteration in a_iter_save else ' '
                loss = train_loss

            text = \
                '%0.5f  %5.3f%s %4.2f  | '%(rate, iteration/10000, asterisk, epoch,) +\
                '%4.3f  %4.3f  %4.3f  %4.3f  | '%(*valid_loss,) +\
                '%4.3f  %4.3f  %4.3f  | '%(*loss,) +\
                '%s' % (time_to_str(timer() - start_timer,'min'))

            return text

        #----
        valid_loss = np.zeros(4,np.float32)
        train_loss = np.zeros(3,np.float32)
        batch_loss = np.zeros_like(train_loss)
        sum_train_loss = np.zeros_like(train_loss)
        sum_train = 0
        loss0 = torch.FloatTensor([0]).cuda().sum()
        loss1 = torch.FloatTensor([0]).cuda().sum()
        loss2 = torch.FloatTensor([0]).cuda().sum()


        start_timer = timer()
        iteration = start_iteration
        epoch = start_epoch
        rate = 0
        while  iteration < num_iteration:
            for t, batch in enumerate(train_loader):

                if (iteration % iter_valid == 0):
                    #if iteration!=start_iteration:
                        valid_loss = do_valid(net, valid_loader)  #
                        pass


                if best_map < valid_loss[1]:
                  best_map = valid_loss[1]
                  a_iter_save.append(iteration)
                  torch.save({
                            'state_dict': net.state_dict(),
                            'iteration': iteration,
                            'epoch': epoch,
                        }, out_dir + '/checkpoint/best_model.pth')
                  pass

                if (iteration % iter_log == 0):
                    print('\r', end='', flush=True)
                    log.write(message(mode='log') + '\n')


                # learning rate schduler ------------
                rate = get_learning_rate(optimizer)

                # one iteration update  -------------
                batch_size = len(batch['index'])
                image = batch['image'].cuda()
                truth_mask = batch['mask'].cuda()
                truth_mask = F.interpolate(truth_mask, size=MASK_SIZE, mode='bilinear', align_corners=False)
                onehot = batch['onehot'].cuda()
                label = onehot.argmax(-1)

                #----
                net.train()
                optimizer.zero_grad()

                if is_mixed_precision:
                    with amp.autocast():
                        logit, mask = data_parallel(net, image)
                        loss0 = F.cross_entropy(logit, label)
                        loss1 = 4*F.binary_cross_entropy_with_logits(mask, truth_mask)

                    #scaler.scale(loss0).backward()
                    #scaler.scale(loss1).backward()
                    scaler.scale(loss0+loss1).backward()
                    scaler.unscale_(optimizer)
                    #torch.nn.utils.clip_grad_norm_(net.parameters(), 2)
                    scaler.step(optimizer)
                    scaler.update()


                else :
                    assert(False)
                    print('fp32')
                    logit, mask = data_parallel(net, image)
                    loss0 = F.cross_entropy(logit, label)
                    loss1 = F.binary_cross_entropy_with_logits(mask, truth_mask.shape)

                    (loss0+loss1).backward()
                    optimizer.step()


                # print statistics  --------
                epoch += 1 / len(train_loader)
                iteration += 1

                batch_loss = np.array([loss0.item(), loss1.item(), loss2.item()])
                sum_train_loss += batch_loss
                sum_train += 1
                if iteration % 100 == 0:
                    train_loss = sum_train_loss / (sum_train + 1e-12)
                    sum_train_loss[...] = 0
                    sum_train = 0

                print('\r', end='', flush=True)
                print(message(mode='print'), end='', flush=True)


        log.write('\n')

In [15]:
run_train()


--- [START 2021-07-07_07-34-10] ----------------------------------------------------------------

	@common.py:  
	pytorch
		seed = 1625643261
		torch.__version__              = 1.9.0+cu102
		torch.version.cuda             = 10.2
		torch.backends.cudnn.version() = 7605
		os['CUDA_VISIBLE_DEVICES']     = 0
		torch.cuda.device_count()      = 1
		torch.cuda.get_device_properties() = (name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


	expirement = SZ640_D201_MASK_0FOLD_WL4
	out_dir  = /content/drive/My Drive/kaggle/covid19-det/output/SZ640_D201_MASK_0FOLD_WL4/fold0-fine

train_dataset : 
	len = 4980
	df  = (4980, 10)
	label distribution
		 0     Negative for Pneumonia:  1366 (0.2743)
		 1         Typical Appearance:  2367 (0.4753)
		 2   Indeterminate Appearance:   864 (0.1735)
		 3        Atypical Appearance:   383 (0.0769)

valid_dataset : 
	len = 1247
	df  = (1247, 10)
	label distribution
		 0     Negative for Pneumonia:   343 (0.2751)
		 1